In [3]:
!head -n 5 transfusion.zip

6��]ܦ�ˢ!�����r☼�wI��$I�<�!R�S�죾1
�U9���U�G>�N_��
n-�P����,8vh��5���|���W��zd��UL�:Ikw��KH�|u�B%��S�V74��!X�Υ�'�VтgUn1������ :�g�x�l�\_YMv�%ZQ&5�c�f������=X-j�0dV�W�kp}gu���r��y�A�K�	<#��hm�:����{'��c�&�"f�<@g��^�6�,��J����3�ȠɨFs1�w�"4e,��*-������HDnۓ��B�Fi��K��x�J"�7hZ6�vH1��67b(;�ȧ5O�$;�h�'_*�U"��%r���^����Z�ߩq&�c��Z�h����Q��+Dr��P���5o('�M.�Hc��9z�:
�(}�SuCķ���*�) ��*j\x^���PK     ̐Q��OQ	  +2   $               transfusion.data
         |�2�k�|�2�k�y?�2�k�PK      b   	    

In [4]:
import pandas as pd
transfusion = pd.read_csv('transfusion.zip')
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [5]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [6]:
transfusion.rename(
    columns={'whether he/she donated blood in March 2007': 'target'},
    inplace=True
)
transfusion.head(3)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1


In [7]:
transfusion.target.value_counts(normalize=True)

0    0.762032
1    0.237968
Name: target, dtype: float64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    transfusion.drop(columns='target'),
    transfusion.target,
    test_size=0.30,
    random_state=40,
    stratify=transfusion.target
)

# Print out the first 3 rows of X_train
X_train.head(3)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
454,21,1,250,21
697,14,3,750,35
688,14,3,750,33


In [9]:
pip install "TPOT==0.11.1"

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Import TPOTClassifier and roc_auc_score
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

# Instantiate TPOTClassifier
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

# AUC score for tpot model
tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

# Print best pipeline steps
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}. {transform}')

Generation 1 - Current best internal CV score: 0.7434181434599157
Generation 2 - Current best internal CV score: 0.7434181434599157
Generation 3 - Current best internal CV score: 0.744089082278481
Generation 4 - Current best internal CV score: 0.744089082278481
Generation 5 - Current best internal CV score: 0.7448371044303798

Best pipeline: MultinomialNB(Normalizer(input_matrix, norm=l2), alpha=0.001, fit_prior=True)

AUC score: 0.7640

Best pipeline steps:
1. Normalizer()
2. MultinomialNB(alpha=0.001)


In [11]:
# X_train's variance, rounding the output to 3 decimal places
X_train.var().round(3)

Recency (months)              71.723
Frequency (times)             36.411
Monetary (c.c. blood)    2275705.479
Time (months)                582.097
dtype: float64

In [12]:
# Import numpy
import numpy as np

# Copy X_train and X_test into X_train_normed and X_test_normed
X_train_normed, X_test_normed = X_train.copy(), X_test.copy()

# Specify which column to normalize
col_to_normalize = 'Monetary (c.c. blood)'

# Log normalization
for df_ in [X_train_normed, X_test_normed]:
    # Add log normalized column
    df_['monetary_log'] = np.log(df_[col_to_normalize])
    # Drop the original column
    df_.drop(columns=col_to_normalize, inplace=True)

# Check the variance for X_train_normed
X_train_normed.var().round(3)

Recency (months)      71.723
Frequency (times)     36.411
Time (months)        582.097
monetary_log           0.839
dtype: float64

In [13]:
# Importing modules
from sklearn import linear_model

# Instantiate LogisticRegression
logreg = linear_model.LogisticRegression(
    solver='liblinear',
    random_state=42
)

# Train the model
logreg.fit(X_train_normed, y_train)

# AUC score for tpot model
logreg_auc_score = roc_auc_score(y_test, logreg.predict_proba(X_test_normed)[:, 1])
print(f'\nAUC score: {logreg_auc_score:.4f}')


AUC score: 0.7755


In [14]:
# Importing itemgetter
from operator import itemgetter

# Sort models based on their AUC score from highest to lowest
sorted(
    [('tpot', tpot_auc_score), ('logreg', logreg_auc_score)],
    key=itemgetter(1),
    reverse=True)

[('logreg', 0.7755035737491878), ('tpot', 0.7640242581763049)]

In [15]:
#saving model
import pickle 
pickle.dump(logreg, open("saved_model.pkl", "wb"))